# Part 3 - Text analysis and ethics

# 3.a Computing PMI

In this assessment you are tasked to discover strong associations between concepts in Airbnb reviews. The starter code we provide in this notebook is for orientation only. The below imports are enough to implement a valid answer.

### Imports, data loading and helper functions

We first connect our google drive, import pandas, numpy and some useful nltk and collections modules, then load the dataframe and define a function for printing the current time, useful to log our progress in some of the tasks.

In [1]:
import pandas as pd
from nltk.tag import pos_tag
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()

In [2]:
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# load stopwords
sw = set(stopwords.words('english'))

In [4]:
basedir = os.getcwd()
df = pd.read_csv(os.path.join(basedir,'reviews.csv'))
# deal with empty reviews
df.comments = df.comments.fillna('')

In [5]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


In [6]:
df.shape

(452143, 6)

### 3.a1 - Process reviews

What to implement: A `function process_reviews(df)` that will take as input the original dataframe and will return it with three additional columns: `tokenized`, `tagged` and `lower_tagged`.

In [7]:
def process_reviews(df):
  tokenized = []
  tagged = []
  lower_tagged = []

  mylen = len(df)
  count = 0
  for index, row in df.iterrows():
    token = word_tokenize(row.comments)
    tokenized.append(token)
    tagged.append(pos_tag(token))
    lower_tagged.append(list(set(pos_tag([item.lower() for item in token]))))
    count += 1
    print(f'{count} out of {mylen}')
    if count % 1000 == 0:      
      break

  df['tokenized'] = tokenized
  df['tagged'] = tagged
  df['lower_tagged'] = lower_tagged


  # df['tokenized'] = [ word_tokenize(row.comments) for index, row in df.iterrows()]
  # print('Tokenizing done!\n')
  # df['tagged'] = [pos_tag(row.tokenized) for index, row in df.iterrows()]
  # print('Tagging done!\n')
  # df['lower_tagged'] = list(set([pos_tag([item.lower() for item in row.tokenized]) for index, row in df.iterrows()]))
  # print('Lower tagging done!\n')
  return df

In [8]:
df = process_reviews(df[:1000])

1 out of 1000
2 out of 1000
3 out of 1000
4 out of 1000
5 out of 1000
6 out of 1000
7 out of 1000
8 out of 1000
9 out of 1000
10 out of 1000
11 out of 1000
12 out of 1000
13 out of 1000
14 out of 1000
15 out of 1000
16 out of 1000
17 out of 1000
18 out of 1000
19 out of 1000
20 out of 1000
21 out of 1000
22 out of 1000
23 out of 1000
24 out of 1000
25 out of 1000
26 out of 1000
27 out of 1000
28 out of 1000
29 out of 1000
30 out of 1000
31 out of 1000
32 out of 1000
33 out of 1000
34 out of 1000
35 out of 1000
36 out of 1000
37 out of 1000
38 out of 1000
39 out of 1000
40 out of 1000
41 out of 1000
42 out of 1000
43 out of 1000
44 out of 1000
45 out of 1000
46 out of 1000
47 out of 1000
48 out of 1000
49 out of 1000
50 out of 1000
51 out of 1000
52 out of 1000
53 out of 1000
54 out of 1000
55 out of 1000
56 out of 1000
57 out of 1000
58 out of 1000
59 out of 1000
60 out of 1000
61 out of 1000
62 out of 1000
63 out of 1000
64 out of 1000
65 out of 1000
66 out of 1000
67 out of 1000
68 o

In [9]:
df

,listing_id,id,date,reviewer_id,reviewer_name,comments,tokenized,tagged,lower_tagged
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...,"[Daniel, is, really, cool, ., The, place, was,...","[(Daniel, NNP), (is, VBZ), (really, RB), (cool...","[(in, IN), (didnt, VBP), (any, DT), (clean, JJ..."
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...,"[Daniel, is, the, most, amazing, host, !, His,...","[(Daniel, NNP), (is, VBZ), (the, DT), (most, R...","[(please, VBP), (way, NN), (bed, NN), (if, IN)..."
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,"[We, had, such, a, great, time, in, Amsterdam,...","[(We, PRP), (had, VBD), (such, JJ), (a, DT), (...","[(bathroom, NN), (10-15, JJ), (in, IN), (we, P..."
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...,"[Very, professional, operation, ., Room, is, v...","[(Very, RB), (professional, JJ), (operation, N...","[(which, WDT), (clean, JJ), (comfortable, JJ),..."
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...,"[Daniel, is, highly, recommended, ., He, provi...","[(Daniel, NNP), (is, VBZ), (highly, RB), (reco...","[(way, NN), (in, IN), (which, WDT), (recommend..."
...,...,...,...,...,...,...,...,...,...
995,28871,257474270,2018-04-26,19146357,PPep,"Highly recommend , Nice place , Near everythin...","[Highly, recommend, ,, Nice, place, ,, Near, e...","[(Highly, NNP), (recommend, NN), (,, ,), (Nice...","[(near, IN), (time, NN), (,, ,), (see, VBP), (..."
996,28871,258531725,2018-04-29,46895354,James,Edwin's place was fantastic for my wife's and ...,"[Edwin, 's, place, was, fantastic, for, my, wi...","[(Edwin, NNP), ('s, POS), (place, NN), (was, V...","[(bathroom, NN), (thank, NN), (in, IN), (being..."
997,28871,259622953,2018-05-01,25353493,Brian,Our stay at Edwin’s was incredible. He is such...,"[Our, stay, at, Edwin, ’, s, was, incredible, ...","[(Our, PRP$), (stay, NN), (at, IN), (Edwin, NN...","[(everywhere, RB), (s, NN), (accommodating, VB..."
998,28871,261853402,2018-05-07,181166673,Manuel Alejandro,"Edwin is such a kind host, he is aware of you ...","[Edwin, is, such, a, kind, host, ,, he, is, aw...","[(Edwin, NNP), (is, VBZ), (such, JJ), (a, DT),...","[(in, IN), (comfortable, JJ), (of, IN), (a, DT..."


In [24]:
# num = 0
# print(len(df.tagged[num]), len(set(df.lower_tagged[num])))
# list(set(df.lower_tagged[num]))

In [12]:
df.lower_tagged[0][1][1][0]

'V'

### 3.a2 - Create a vocabulary

What to implement: A function `get_vocab(df)` which takes as input the DataFrame generated in step 1.c, and returns two lists, one for the 1,000 most frequent center words (nouns) and one for the 1,000 most frequent context words (either verbs or adjectives). 

In [31]:
def get_vocab(df):
  cent_list, cont_list = [], []

  for review in df.lower_tagged:
    cent_list.extend([word for word in [list_of_words[0] for list_of_words in review if list_of_words[1][0] == 'N']])
    cont_list.extend([word for word in [list_of_words[0] for list_of_words in review if (list_of_words[1][0] == 'J') or (list_of_words[1][0] == 'V')]])
    
  cent_dict = Counter(cent_list)
  cont_dict = Counter(cont_list)

  cent_vocab = [key for key, value in sorted(cent_dict.items(), key=lambda item: item[1])][:1000]
  cont_vocab = [key for key, value in sorted(cont_dict.items(), key=lambda item: item[1])][:1000]

  return cent_vocab, cont_vocab

In [32]:
cent_vocab, cont_vocab = get_vocab(df)

In [48]:
cent_vocab[0]

'planet'

### 3.a3 Count co-occurrences between center and context words

What to implement: A function `get_coocs(df, center_vocab, context_vocab)` which takes as input the DataFrame generated in step 1, and the lists generated in step 2 and returns a dictionary of dictionaries, of the form in the example above. It is up to you how you define context (full review? per sentence? a sliding window of fixed size?), and how to deal with exceptional cases (center words occurring more than once, center and context words being part of your vocabulary because they are frequent both as a noun and as a verb, etc). Use comments in your code to justify your approach. 

In [33]:
comments = df.comments
type(comments)

pandas.core.series.Series

In [34]:
comments[:5]

0    Daniel is really cool. The place was nice and ...
1    Daniel is the most amazing host! His place is ...
2    We had such a great time in Amsterdam. Daniel ...
3    Very professional operation. Room is very clea...
4    Daniel is highly recommended.  He provided all...
Name: comments, dtype: object

In [58]:
def get_coocs(df, cent_vocab, cont_vocab):
  sentences = []
  comments = df.comments

  for comment in comments:
    sentences.extend([sentence for sentence in comment.split('.')])
  
  # print(sentences)
  
  coocs = {}

  for center_word in cent_vocab:
    words = []
    for sentence in sentences:
      if center_word in sentence:
        words_in_sentence = word_tokenize(sentence)
        words.extend([word for word in words_in_sentence if word in cont_vocab])
    
    center_word_dict = dict(Counter(words))
    coocs[center_word] = center_word_dict
    
  # cent_dict = Counter(cent_list)
  # cont_dict = Counter(cont_list)
  
  return coocs  

In [61]:
coocs = get_coocs(df, cent_vocab, cont_vocab)

In [66]:
coocs['planet']['lonely']

1

### 3.a4 Convert co-occurrence dictionary to 1000x1000 dataframe
What to implement: A function called `cooc_dict2df(cooc_dict)`, which takes as input the dictionary of dictionaries generated in step 3 and returns a DataFrame where each row corresponds to one center word, and each column corresponds to one context word, and cells are their corresponding co-occurrence value. Some (x,y) pairs will never co-occur, you should have a 0 value for those cases. 

In [77]:
def cooc_dict2df(coocs):
  coocdf = pd.DataFrame(columns=cont_vocab, index = cent_vocab)

  for index, row in coocdf.iterrows():
    for word in cont_vocab:
      try:
        coocdf[word][index] = coocs[index][word]
      except: 
        coocdf[word][index] = 0

  return coocdf

In [78]:
coocdf = cooc_dict2df(coocs)
coocdf.shape

(1000, 1000)

### 3.a5 Raw co-occurrences to PMI scores

What to implement: A function `cooc2pmi(df)` that takes as input the DataFrame generated in step 4, and returns a new DataFrame with the same rows and columns, but with PMI scores instead of raw co-occurrence counts. 

In [ ]:
def cooc2pmi(df):
  # your code here
  return pmidf

In [ ]:
pmidf = cooc2pmi(coocdf)
pmidf.shape

### 3.a6 Retrieve top-k context words, given a center word

What to implement: A function `topk(df, center_word, N=10)` that takes as input: (1) the DataFrame generated in step 5, (2) a `center_word` (a string like `‘towels’`), and (3) an optional named argument called `N` with default value of 10; and returns a list of `N` strings, in order of their PMI score with the `center_word`. You do not need to handle cases for which the word `center_word` is not found in `df`. 

In [ ]:
def topk(df, center_word, N=10):
  # your code here
  return top_words

In [ ]:
topk(pmidf, 'coffee')

# 3.b Ethical, social and legal implications



Local authorities in touristic hotspots like Amsterdam, NYC or Barcelona regulate the price of recreational apartments for rent to, among others, ensure that fair rent prices are kept for year-long residents. Consider your price recommender for hosts in Question 2c. Imagine that Airbnb recommends a new host to put the price of your flat at a price which is above the official regulations established by the local government. Upon inspection, you realize that the inflated price you have been recommended comes from many apartments in the area only being offered during an annual event which brings many tourists, and which causes prices to rise. 

In this context, critically reflect on the compliance of this recommender system with **one of the five actions** outlined in the **UK’s Data Ethics Framework**. You should prioritize the action that, in your opinion, is the weakest. Then, justify your choice by critically analyzing the three **key principles** outlined in the Framework, namely _transparency_, _accountability_ and _fairness_. Finally, you should propose and critically justify a solution that would improve the recommender system in at least one of these principles. You are strongly encouraged to follow a scholarly approach, e.g., with peer-reviewed references as support. 

Your report should be between 500 and 750 words long.  

### Your answer here. No Python, only Markdown.

Write your answer after the line.

---

...